In [ ]:
from flask import Flask, render_template, request
import csv
import xml.etree.ElementTree as ET

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/convert', methods=['POST'])
def convert():
    kml_file = request.files['file']

    try:
        tree = ET.parse(kml_file)
        root = tree.getroot()

        placemarks = root.findall('.//{http://www.opengis.net/kml/2.2}Placemark')

        data = []
        for placemark in placemarks:
            data.append(extract_placemark_data(placemark))

        return render_template('output.html', data=data)
    except Exception as e:
        return f"An error occurred: {str(e)}"

def extract_placemark_data(placemark):
    name_element = placemark.find('.//{http://www.opengis.net/kml/2.2}name')
    name = name_element.text if name_element is not None else ''

    coordinates_element = placemark.find('.//{http://www.opengis.net/kml/2.2}Point/{http://www.opengis.net/kml/2.2}coordinates')
    coordinates = coordinates_element.text.strip().split(',') if coordinates_element is not None else ['', '', '']

    longitude, latitude, _ = coordinates
    latitude = format(float(latitude), '.6f')
    longitude = format(float(longitude), '.6f')
    return name, latitude, longitude

if __name__ == '__main__':
    app.run()
